## Example of using `VaspRelaxationWorkChain`

In [1]:
%load_ext aiida
%aiida

Loaded AiiDA DB environment - profile name: 2.0-dev.

In [2]:
from aiida_user_addons.tools.pymatgen import load_mp_struct
from aiida.orm import StructureData
from aiida_user_addons.common.inputset.vaspsets import VASPInputSet
from aiida_user_addons.tools.dryrun import dryrun_vasp
from aiida_user_addons.vworkflows.relax import VaspRelaxWorkChain
from pprint import pprint

Just get a SrTiO3 structure from materials project

In [3]:
sto = load_mp_struct("mp-5229")

In [4]:
sto.get_pymatgen()

Structure Summary
Lattice
    abc : 3.94513 3.94513 3.94513
 angles : 90.0 90.0 90.0
 volume : 61.40220340476369
      A : 3.94513 0.0 0.0
      B : 0.0 3.94513 0.0
      C : 0.0 0.0 3.94513
    pbc : True True True
PeriodicSite: Sr (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Ti (1.9726, 1.9726, 1.9726) [0.5000, 0.5000, 0.5000]
PeriodicSite: O (1.9726, 0.0000, 1.9726) [0.5000, 0.0000, 0.5000]
PeriodicSite: O (1.9726, 1.9726, 0.0000) [0.5000, 0.5000, 0.0000]
PeriodicSite: O (0.0000, 1.9726, 1.9726) [0.0000, 0.5000, 0.5000]

Break the symmetry

In [5]:
sto.sites[0].x = 0.1

Load some default input dictionary, note one can customise it using the `overrides` settings.  
Here we use `VaspRelaxUpdater` which is designed for quick setup of `ProcessBuilder` for `VaspRelaxWorkChain`.

These sets are for convenience only - the full input is always recorded by AiiDA.

In [6]:
from aiida_user_addons.common.builder_updater import VaspRelaxUpdater

In [7]:
builder = VaspRelaxWorkChain.get_builder()
upd = VaspRelaxUpdater(builder)

# The actual calculations parameters are set under the `vasp` input port
upd.use_inputset(sto, "UCLRelaxSet", overrides={'encut': 400, 'lorbit': None})
upd.set_code(Code.get_from_string("vasp-std@localhost"))
upd.set_kspacing(0.05) # In unit of A^1 * 2pi
upd.set_label('STO broken')
upd.update_options(max_wallclock_seconds=3600)
upd.set_resources(num_machines=1, tot_num_mpiprocs=2)

Using input set file at: /home/bonan/aiida_envs/aiida-2.0-dev/aiida-user-addons/aiida_user_addons/common/inputset/UCLRelaxSet.yaml


/home/bonan/aiida_envs/aiida-2.0-dev/aiida-user-addons/aiida_user_addons/common/builder_updater.py:267: UserWarning: Using default options template - adjustment needed for the target computer
  warn(


Full inputs can be peek by checking the `.builder` attribute

In [8]:
upd.builder

Process class: VaspRelaxWorkChain
Inputs:
metadata:
  label: STO broken
structure: O3SrTi
vasp:
  code: ''
  dynamics: {}
  kpoints_spacing: 0.05
  metadata: {}
  options:
    import_sys_environment: false
    max_wallclock_seconds: 3600
    resources:
      num_machines: 1
      tot_num_mpiprocs: 2
  parameters:
    incar:
      algo: normal
      ediff: 4.9999999999999996e-06
      encut: 400
      gga: ps
      ismear: 0
      ispin: 2
      lasph: true
      ldau: true
      ldauj:
      - 0.0
      - 0.0
      - 0.0
      ldaul:
      - -1
      - 2
      - -1
      ldautype: 2
      ldauu:
      - 0.0
      - 4.0
      - 0.0
      lmaxmix: 4
      lreal: false
      lvhar: true
      lwave: false
      nedos: 2000
      nelm: 200
      nelmin: 4
      nwrite: 1
      prec: accurate
      sigma: 0.05
  potential_family: PBE.54
  potential_mapping:
    O: O
    Sr: Sr_sv
    Ti: Ti_pv


Set the relaxation settings  
The provides some default parameters to control the relaxation

In [9]:
upd.update_relax_settings(convergence_on=True, 
                             # Converge between the input and output structure, need when we expect large cell colume change to ensure basis set consistency
                             # The default is 'last', which will check against the last two structures
                             convergence_mode='inout'   
                            )

A new `relax_settings` field should be configured in the builder

In [10]:
upd.builder

Process class: VaspRelaxWorkChain
Inputs:
metadata:
  label: STO broken
relax_settings:
  algo: cg
  clean_reuse: true
  convergence_absolute: false
  convergence_max_iterations: 5
  convergence_mode: inout
  convergence_on: true
  convergence_positions: 0.1
  convergence_shape_angles: 0.1
  convergence_shape_lengths: 0.1
  convergence_volume: 0.01
  force_cutoff: 0.03
  keep_magnetization: false
  keep_sp_workdir: false
  perform: true
  positions: true
  reuse: false
  shape: true
  steps: 60
  volume: true
structure: O3SrTi
vasp:
  code: ''
  dynamics: {}
  kpoints_spacing: 0.05
  metadata: {}
  options:
    import_sys_environment: false
    max_wallclock_seconds: 3600
    resources:
      num_machines: 1
      tot_num_mpiprocs: 2
  parameters:
    incar:
      algo: normal
      ediff: 4.9999999999999996e-06
      encut: 400
      gga: ps
      ismear: 0
      ispin: 2
      lasph: true
      ldau: true
      ldauj:
      - 0.0
      - 0.0
      - 0.0
      ldaul:
      - -1
      

Finally we submit the calculation 

`submit` does not run the calculation but instead it serialise all inputs of the calculation to the storage and mark the process to be run.  
The *daemon* will pick up the job and actually run it. The *daemon* can be started with `verdi daemon start`...

In [11]:
from aiida.engine import submit

In [12]:
running = submit(builder)
running

<WorkChainNode: uuid: eda79793-c3f9-42bd-ab67-40b94a8902ac (pk: 1006) (aiida.workflows:vaspu.relax)>

## Analysis

load the node using UUID - this ensures that if we restart the notebook we can carry on with the analysis

In [17]:
work = load_node("eda79793")  

Verbose report emitted while the procss was running - we can see that the workchain checked the convergence and resubmit the job if not happy.  
It will also do a final calculation to ensure the energy is accurate - VASP can only do constant-basis relaxations. 

In [18]:
!verdi process report {work.id}

2023-02-17 16:49:41 [78 | REPORT]: [1006|VaspRelaxWorkChain|run_relax]: launching VaspWorkChain<1009> iterations #1
2023-02-17 16:49:42 [79 | REPORT]:   [1009|VaspWorkChain|run_process]: launching VaspCalculation<1012> iteration #1


In [19]:
work.inputs.structure.get_cell_volume()

61.40220340476369

In [21]:
work.outputs.relax__structure.get_cell_volume()

59.19834628391486

In [22]:
work.outputs.misc.get_dict()

{'version': '6.2.0',
 'run_stats': {'user_time': 17.406,
  'system_time': 0.27,
  'elapsed_time': 17.777,
  'mem_usage_base': 30000.0,
  'mem_usage_grid': 9734.0,
  'mem_usage_wavefun': 9717.0,
  'mem_usage_fftplans': 3139.0,
  'average_memory_used': None,
  'maximum_memory_used': 102836.0,
  'mem_usage_nonl-proj': 9661.0,
  'total_cpu_time_used': 17.675,
  'mem_usage_one-center': 155.0},
 'run_status': {'nsw': 0,
  'nelm': 200,
  'nbands': 26,
  'finished': True,
  'ionic_converged': None,
  'contains_nelm_breach': False,
  'electronic_converged': True,
  'last_iteration_index': [1, 15],
  'consistent_nelm_breach': False},
 'maximum_force': 0.0,
 'notifications': [],
 'maximum_stress': 0.34670663,
 'total_energies': {'energy_extrapolated': -38.94422687,
  'energy_extrapolated_electronic': -38.94422687}}

## Provenance graph example

In [23]:
from aiida.tools.visualization import Graph

g = Graph()
g.recurse_descendants(work)
g.recurse_ancestors(work, depth=1)

In [24]:
g.graphviz.render('relax', format='png')

'relax.png'

![Relaxation Provenance](relax.png)